### Pyspark
#### LogisticRegression

In [0]:
#Pyspark-Logistic Regression
#import the required libararies
import pyspark
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import col,count, when
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.feature import StandardScaler
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler

In [0]:
#Load and read the data
df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/praneethsanthosh555@gmail.com/diabetes.csv",header=True)
df1.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
 6| 148| 72| 35| 0|33.6| 0.627| 50| 1|
 1| 85| 66| 29| 0|26.6| 0.351| 31| 0|
 8| 183| 64| 0| 0|23.3| 0.672| 32| 1|
 1| 89| 66| 23| 94|28.1| 0.167| 21| 0|
 0| 137| 40| 35| 168|43.1| 2.288| 33| 1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows

In [0]:
df1.printSchema()

root
-- Pregnancies: string (nullable = true)
-- Glucose: string (nullable = true)
-- BloodPressure: string (nullable = true)
-- SkinThickness: string (nullable = true)
-- Insulin: string (nullable = true)
-- BMI: string (nullable = true)
-- DiabetesPedigreeFunction: string (nullable = true)
-- Age: string (nullable = true)
-- Outcome: string (nullable = true)

In [0]:
new_data=df1.select(*(col(c).cast("float").alias(c) for c in df1.columns))
new_data.printSchema()

root
-- Pregnancies: float (nullable = true)
-- Glucose: float (nullable = true)
-- BloodPressure: float (nullable = true)
-- SkinThickness: float (nullable = true)
-- Insulin: float (nullable = true)
-- BMI: float (nullable = true)
-- DiabetesPedigreeFunction: float (nullable = true)
-- Age: float (nullable = true)
-- Outcome: float (nullable = true)

In [0]:
#Check any null values are present in the data
data_with_null= df1.select([count(when(col(c).isNull(),c)).alias(c) for c in df1.columns]).show()
data_with_null

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+

In [0]:
#Split the data 
col=new_data.columns
col.remove("Outcome")
assembler=VectorAssembler(inputCols=col,
                          outputCol="features"
                         )

In [0]:
#Let's use the transform method to transform our dataset
data = assembler.transform(new_data)
data.select("features",'Outcome').show(truncate=False)

+-----------------------------------------------------------------------+-------+
features |Outcome|
+-----------------------------------------------------------------------+-------+
[6.0,148.0,72.0,35.0,0.0,33.599998474121094,0.6269999742507935,50.0] |1.0 |
[1.0,85.0,66.0,29.0,0.0,26.600000381469727,0.35100001096725464,31.0] |0.0 |
[8.0,183.0,64.0,0.0,0.0,23.299999237060547,0.671999990940094,32.0] |1.0 |
[1.0,89.0,66.0,23.0,94.0,28.100000381469727,0.16699999570846558,21.0] |0.0 |
[0.0,137.0,40.0,35.0,168.0,43.099998474121094,2.2880001068115234,33.0] |1.0 |
[5.0,116.0,74.0,0.0,0.0,25.600000381469727,0.20100000500679016,30.0] |0.0 |
[3.0,78.0,50.0,32.0,88.0,31.0,0.24799999594688416,26.0] |1.0 |
[10.0,115.0,0.0,0.0,0.0,35.29999923706055,0.1340000033378601,29.0] |0.0 |
[2.0,197.0,70.0,45.0,543.0,30.5,0.15800000727176666,53.0] |1.0 |
[8.0,125.0,96.0,0.0,0.0,0.0,0.23199999332427979,54.0] |1.0 |
[4.0,110.0,92.0,0.0,0.0,37.599998474121094,0.19099999964237213,30.0] |0.0 |
[10.0,168.0,74.0,0.0,0.0,38.0,0.5370000004768372,34.0] |1.0 |
[10.0,139.0,80.0,0.0,0.0,27.100000381469727,1.440999984741211,57.0] |0.0 |
[1.0,189.0,60.0,23.0,846.0,30.100000381469727,0.39800000190734863,59.0]|1.0 |
[5.0,166.0,72.0,19.0,175.0,25.799999237060547,0.5870000123977661,51.0] |1.0 |
[7.0,100.0,0.0,0.0,0.0,30.0,0.48399999737739563,32.0] |1.0 |
[0.0,118.0,84.0,47.0,230.0,45.79999923706055,0.5509999990463257,31.0] |1.0 |
[7.0,107.0,74.0,0.0,0.0,29.600000381469727,0.2540000081062317,31.0] |1.0 |
[1.0,103.0,30.0,38.0,83.0,43.29999923706055,0.18299999833106995,33.0] |0.0 |
[1.0,115.0,70.0,30.0,96.0,34.599998474121094,0.5289999842643738,32.0] |1.0 |
+-----------------------------------------------------------------------+-------+
only showing top 20 rows

In [0]:
#Scale down all the values in the data
standardScaler = StandardScaler().setInputCol("features").setOutputCol("scaled_features")
data=standardScaler.fit(data).transform(data)

In [0]:
assemble_data= data.select("scaled_features","Outcome")
assemble_data.show(5)

+--------------------+-------+
 scaled_features|Outcome|
+--------------------+-------+
[1.78063837321943...| 1.0|
[0.29677306220323...| 0.0|
[2.37418449762590...| 1.0|
[0.29677306220323...| 0.0|
[0.0,4.2849165233...| 1.0|
+--------------------+-------+
only showing top 5 rows

In [0]:
#Split the data in to train and test data
train_df,test_df=assemble_data.randomSplit([0.7,0.3])

In [0]:
train_df.show(5)
test_df.show(5)

+--------------------+-------+
 scaled_features|Outcome|
+--------------------+-------+
(8,[0,1,6,7],[0.5...| 0.0|
(8,[0,1,6,7],[0.8...| 0.0|
(8,[0,1,6,7],[1.7...| 0.0|
(8,[0,1,6,7],[2.0...| 0.0|
(8,[1,5,6,7],[2.2...| 0.0|
+--------------------+-------+
only showing top 5 rows

+--------------------+-------+
 scaled_features|Outcome|
+--------------------+-------+
(8,[0,1,6,7],[0.5...| 0.0|
(8,[0,1,6,7],[2.9...| 1.0|
(8,[1,5,6,7],[3.0...| 0.0|
(8,[1,5,6,7],[3.6...| 0.0|
(8,[1,5,6,7],[3.7...| 1.0|
+--------------------+-------+
only showing top 5 rows

In [0]:
log_reg=LogisticRegression(featuresCol="scaled_features",labelCol="Outcome",maxIter=10)
log_model=log_reg.fit(train_df)

In [0]:
#Get the predictions
prediction=log_model.transform(test_df)

In [0]:
prediction.select("prediction","Outcome").show(10)

+----------+-------+
prediction|Outcome|
+----------+-------+
 0.0| 0.0|
 0.0| 1.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
+----------+-------+
only showing top 10 rows

In [0]:
#Get the results or performance on the test data
#Compute raw scores on the test dataset
pred=prediction.select("Outcome","prediction").rdd.map(lambda row: row[0:])

In [0]:
metrics=BinaryClassificationMetrics(pred)

#Area under the curve
print("Area under ROC = %s" % metrics.areaUnderROC)
print("Area under PR = %s" % metrics.areaUnderPR)

Area under ROC = 0.8064327485380117
Area under PR = 0.5659740259740259